# < Contents Type Predict >

In [ ]:
# 경고 메시지 무시
import warnings
warnings.filterwarnings("ignore")

## 1. 데이터 로드

In [ ]:
import pandas as pd

# 데이터 로드
raw = pd.read_excel('data/lilybyred_무스틴트_v1.xlsx')

# 데이터 확인
raw[:3]

,keyword,site,date,title,username,content,click,link
0,릴리바이레드 무스틴트,naverBlog,2019-08-01,미샤 트리플 섀도우 8호 오렌지 퍼레이드,왼쪽정강이,미샤 트리플 섀도우 8호 오렌지 퍼레이드​​전에 미샤 워터틴트를 사며 같이 구매했던...,NaN,https://m.blog.naver.com/helena7463/221591237054
1,릴리바이레드 무스틴트,naverBlog,2019-08-01,[공지] 올리브영 웜톤틴트 추천!릴리바이레드 무드 라이어 벨벳 틴트 04 그윽한 홍시인척,이블린,​#릴리바이레드 무드 라이어 벨벳 틴트04 그윽한 홍시인척-본 포스팅은 직접 구매후...,NaN,https://m.blog.naver.com/gksquf9506?Redirect=L...
2,릴리바이레드 무스틴트,naverBlog,2019-08-01,[일상] 당일치기 경주여행,영주,날씨가 너무 더워서 조금만 걸어도 땀이 나는 요즘 오빠랑 시밀러룩 찍은지 너무 오래...,NaN,https://m.blog.naver.com/love960814/221602350784


In [ ]:
# 데이터 선택
data = raw["content"].dropna().reset_index(drop = True)
data

0       미샤 트리플 섀도우 8호 오렌지 퍼레이드​​전에 미샤 워터틴트를 사며 같이 구매했던...
1       ​#릴리바이레드 무드 라이어 벨벳 틴트04 그윽한 홍시인척-본 포스팅은 직접 구매후...
2       날씨가 너무 더워서 조금만 걸어도 땀이 나는 요즘 오빠랑 시밀러룩 찍은지 너무 오래...
3       아임미미 미스터리 볼륨 틴트005 미스터리 비트미미박스 체험단으로 사용해본 제품입니...
4       종강한 나를 반겨주듯이 ​​​​여름이 성큼 다가왔다 ​​​​갑자기 더워진 바람이 낯...
                              ...                        
1035    릴리바이레드 앙큼 라이어 코팅틴트 4g 수줍은 복숭아인척 1개제품 보러가기5720원...
1036    ​벌써 흘러가버린 2020년 1월 한달과 함께참 한결같은 소똥히의 소비패턴 관찰하기...
1037    안녕하세요 뷰티블로거 프롬조제예요 :)색조 장인 릴리바이레드에서 신상 소식이 들려와...
1038     안녕하세요 뷰스타 다랑입니다 :)오늘은 릴리바이레드에서 새롭게 출시하는아이브로우를...
1039    ​​안녕하세요 노루군입니다.한동안 잡지부록을 소개해드린 적이 없었는데이번에 알라딘 ...
Name: content, Length: 1040, dtype: object

## 2. 데이터 전처리
### 1) 텍스트 정제  
**: 숫자, 대소문자 통일, 이모티콘, 링크, 이메일 제거**

In [ ]:
import re

# 텍스트 정제 함수
def preprocessing_text(data):
    url = "(https?://)?(www.)?[a-zA-Z0-9./?=&-_]+[.]?[/][a-zA-Z0-9./?=&-_%]+"
    email = "[-_.+a-zA-Z0-9]+[@].+[.][[a-zA-Z0-9]+|[a-zA-Z0-9]+[.][a-zA-Z0-9]+]"
    
    for i in range(len(data)):
        data[i] = re.sub(url,"", data[i])
        data[i] = re.sub(email,"", data[i])
        data[i] = re.sub("[^가-힣a-zA-Z ]","", data[i])
        data[i] = data[i].lower()
    return data

# 텍스트 정제
clean_text = preprocessing_text(data)

### 2) 형태소 추출 _Mecab

In [ ]:
# 형태소 분석기 Mecab
from konlpy.tag import Mecab
mecab = Mecab()

# mecab.morphs(clean_text[0])

# 명사만 추출
# mecab.nouns(data)

# 품사 포함 추출
# mecab.pos(clean_text[0],flatten=True)

# 형태소 분석
from tqdm import tqdm

corpus = []

for i in tqdm(range(len(clean_text))):
    sen = mecab.nouns(clean_text[i])
    sen = [word for word in sen if len(word) > 1] #1글자 이상만 추출
    corpus.append(sen)

100%|██████████| 1040/1040 [00:03<00:00, 305.23it/s]


**형태소 추출이 끝나면 꼭 형태소 리스트 수와 데이터 수를 비교**  
**형태소 리스트 수와 데이터 수가 같아야 나중에 학습한 이후에 데이터를 통합할 수 있다.**

In [ ]:
len(data), len(corpus)

(1040, 1040)

### 3) 단어 정제  
**: 동음이의어, 불완전하게 추출된 형태소 처리**

In [ ]:
# 바꾸기
clean_corpus = [[re.sub('^리바이$','릴리바이레드',noun_) for noun_ in nouns] for nouns in corpus]

# 제거
clean_corpus = [[noun_ for noun_ in nouns if noun_ != "레드"] for nouns in clean_corpus]
clean_corpus = [[noun_ for noun_ in nouns if noun_ != "틴트"] for nouns in clean_corpus]

# 불용어 삭제
f = open("data/stopwords_list.txt", 'r', encoding='CP949')
line = [line.rstrip() for line in f.readlines()]

for i in range(len(line)):
    clean_corpus = [[noun_ for noun_ in nouns if noun_ != line[i]] for nouns in clean_corpus]

In [ ]:
# 단어 카운트 함수
def count_noun(data):
    t_noun = list()
    
    for _ in data:
        t_noun.extend(_)
    
    t_noun_s = pd.DataFrame(t_noun, columns=['word'])
    t_noun_f = t_noun_s.groupby('word').size().reset_index(name='count').sort_values(['count'], ascending=False).reset_index(drop=True)
    
    return t_noun_f

# 단어 카운트
frq_corpus = count_noun(clean_corpus)
frq_corpus[:10]

,word,count
0,구매,4277
1,컬러,3145
2,이미지,3036
3,페이지,2998
4,클릭,2977
5,제품,2509
6,릴리바이레드,2419
7,사용,1705
8,핑크,1540
9,발색,1423


### 4) 단어 임베딩  
**: 문자를 숫자로 바꾸는 과정**

In [ ]:
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
sequences = tokenizer.texts_to_sequences(corpus)

word_vocab = tokenizer.word_index

MAX_SEQUENCE_LENGTH = 3470
word_vector = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH,padding='post')
word_vector

array([[  197,   298,    22, ...,     0,     0,     0],
       [    9,     3,    47, ...,     0,     0,     0],
       [  432,   153,   262, ...,     0,     0,     0],
       ...,
       [  134,    60,   852, ...,     0,     0,     0],
       [  134,   603, 16704, ...,     0,     0,     0],
       [  134, 16709,  1974, ...,     0,     0,     0]], dtype=int32)

## 3. 데이터 예측
### 1) 타입 예측 모델 불러오기

In [ ]:
from sklearn.externals import joblib
clf = joblib.load('type_modeling.pkl')

### 2) 타입 예측하기

In [ ]:
type_hat = clf.predict(word_vector)

### 3) 예측한 값을 데이터에 넣기

In [ ]:
fin = raw
c_type = {1:"바이럴", 2:"소비자"}
fin['type'] = pd.Series(type_hat).map(c_type)
fin[:3]

,keyword,site,date,title,username,content,click,link,type
0,릴리바이레드 무스틴트,naverBlog,2019-08-01,미샤 트리플 섀도우 8호 오렌지 퍼레이드,왼쪽정강이,미샤 트리플 섀도우 8호 오렌지 퍼레이드​​전에 미샤 워터틴트를 사며 같이 구매했던...,NaN,https://m.blog.naver.com/helena7463/221591237054,소비자
1,릴리바이레드 무스틴트,naverBlog,2019-08-01,[공지] 올리브영 웜톤틴트 추천!릴리바이레드 무드 라이어 벨벳 틴트 04 그윽한 홍시인척,이블린,​#릴리바이레드 무드 라이어 벨벳 틴트04 그윽한 홍시인척-본 포스팅은 직접 구매후...,NaN,https://m.blog.naver.com/gksquf9506?Redirect=L...,소비자
2,릴리바이레드 무스틴트,naverBlog,2019-08-01,[일상] 당일치기 경주여행,영주,날씨가 너무 더워서 조금만 걸어도 땀이 나는 요즘 오빠랑 시밀러룩 찍은지 너무 오래...,NaN,https://m.blog.naver.com/love960814/221602350784,소비자


## 4. 데이터 저장

In [ ]:
fin.to_excel("lilybyred_무스틴트_v1_predict.xlsx", encoding='utf-8', index=False)